### Importations

In [1]:
import os, sys, cv2, numpy as np, matplotlib.pyplot as plt, subprocess
from base64 import b64encode
from IPython.display import HTML

#### Vérification des Versions

In [2]:
# Check the version
print("numpy " + np.__version__)
print("openCV " + cv2.__version__)

numpy 1.26.4
openCV 4.10.0


### Fonctions de Dessin

In [3]:
def drawRectangle(frame, bbox):
    p1 = (int(bbox[0]), int(bbox[1]))
    p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
    cv2.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)

### Dessin de Texte (FPS, Infos sur le Tracker)

In [4]:
def drawText(frame, txt, location, color=(50, 170, 50)):
    cv2.putText(frame, txt, location, cv2.FONT_HERSHEY_SIMPLEX, 1, color, 3)

### Initialisation du Tracker

In [5]:
tracker_types = ["BOOSTING", "MIL", "KCF", "CSRT", "TLD", "MEDIANFLOW", "GOTURN", "MOSSE"]
tracker_type = tracker_types[2]  # Using KCF tracker

if tracker_type == "BOOSTING":
    tracker = cv2.legacy.TrackerBoosting_create()
elif tracker_type == "MIL":
    tracker = cv2.legacy.TrackerMIL_create()
elif tracker_type == "KCF":
    tracker = cv2.TrackerKCF_create()
elif tracker_type == "CSRT":
    tracker = cv2.TrackerCSRT_create()
elif tracker_type == "TLD":
    tracker = cv2.legacy.TrackerTLD_create()
elif tracker_type == "MEDIANFLOW":
    tracker = cv2.legacy.TrackerMedianFlow_create()
elif tracker_type == "GOTURN":
    tracker = cv2.TrackerGOTURN_create()
else:
    tracker = cv2.legacy.TrackerMOSSE_create()

### Initialisation du Subtracteur de Fond

In [6]:
backSub = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=True)

### Configuration de la Vidéo d'Entrée et de Sortie

In [7]:
video_input_file_name = "./video_car_2.mp4"
video = cv2.VideoCapture(video_input_file_name)

if not video.isOpened():
    print("Could not open video")
    sys.exit()

width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

fps_output = 5  # FPS pour la sortie
video_output_file_name = "tracked_multiple_objects.mp4"
video_out = cv2.VideoWriter(video_output_file_name, cv2.VideoWriter_fourcc(*"XVID"), fps_output, (width, height))

### Boucle de Traitement de la Vidéo

In [8]:
while True:
    ok, frame = video.read()
    if not ok:
        break

    # Apply background subtraction to detect moving objects
    fgMask = backSub.apply(frame)

    # Find contours of moving objects
    contours, _ = cv2.findContours(fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw bounding boxes around detected objects
    for contour in contours:
        if cv2.contourArea(contour) > 500:  # Filter small objects (noise)
            (x, y, w, h) = cv2.boundingRect(contour)
            drawRectangle(frame, (x, y, w, h))

    # Draw FPS on the frame
    fps = video.get(cv2.CAP_PROP_FPS)
    drawText(frame, f"FPS: {fps:.2f}", (10, 40))

    # Display the resulting frame
    cv2.imshow("Tracking", frame)

    # Write the frame to output video
    video_out.write(frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
video.release()
video_out.release()
cv2.destroyAllWindows()

### Conversion en Format H.264 avec ffmpeg

In [9]:

output_h264 = f"tracked_{tracker_type}_h264.mp4"
subprocess.run(['ffmpeg', '-y', '-i', video_output_file_name, '-vcodec', 'libx264', output_h264], check=True)

if os.path.exists(output_h264):
    with open(output_h264, "rb") as f:
        mp4 = f.read()
    data_url = f"data:video/mp4;base64,{b64encode(mp4).decode()}"
    HTML(f"""<video width=640 controls><source src="{data_url}" type="video/mp4"></video>""")
else:
    print(f"Error: {output_h264} not found.")